In [ ]:
import urllib.request
import pandas as pd
import json
import re

client_id = "CLIENT_ID"
client_secret = "CLIENT_SECRET"

display = 5
start = 1
end = 50
sort = "comment"

# 사용자 입력 받기
region = input("검색할 지역: ")

# 각 카테고리별 데이터 프레임 생성
local_korean_food_df = pd.DataFrame(columns=("Title", "Link", "Description", "Telephone", "Address"))
local_chinese_food_df = pd.DataFrame(columns=("Title", "Link", "Description", "Telephone", "Address"))
local_japanese_food_df = pd.DataFrame(columns=("Title", "Link", "Description", "Telephone", "Address"))
local_western_food_df = pd.DataFrame(columns=("Title", "Link", "Description", "Telephone", "Address"))

# 태그 제거를 위한 함수
def remove_tags(text):
    return re.sub(re.compile('<.*?>'), '', text)

# 카테고리별 검색 및 데이터 수집 함수
def search_food(category, df):
    query = urllib.parse.quote(f"{region} {category}")
    idx = 0
    for start_index in range(start, end, display):
        url = f"https://openapi.naver.com/v1/search/local?query={query}&display={display}&start={start_index}&sort={sort}"
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200: # 정상적인 응답일 경우
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']
            for item in items:
                title = remove_tags(item['title'])
                link = item['link']
                description = remove_tags(item['description'])
                address = item['address']
                telephone = item['telephone']
                df.loc[idx] = [title, link, description, telephone, address]
                idx += 1
        else:
            print("Error Code:" + rescode)

# 각 카테고리별 검색 실행
search_food("한식", local_korean_food_df)
search_food("중식", local_chinese_food_df)
search_food("일식", local_japanese_food_df)
search_food("양식", local_western_food_df)

# 결과 출력
print(f"{region} 지역의 한식집 검색 결과:")
print(local_korean_food_df)
print(f"\n{region} 지역의 중식집 검색 결과:")
print(local_chinese_food_df)
print(f"\n{region} 지역의 일식집 검색 결과:")
print(local_japanese_food_df)
print(f"\n{region} 지역의 양식집 검색 결과:")
print(local_western_food_df)
